In [194]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [372]:
def linkInfoCrawler(target):
    driver = webdriver.Chrome(executable_path=r'/Users/chenboyu/Desktop/野村投信/chromedriver')
    driver.get('https://www.cnbc.com/search/?query=' + str(target) + '&qsearchterm=' + str(target))
    time.sleep(3)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(1)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            links = driver.find_elements_by_css_selector('a.resultlink')
            if(len(links) >= 4250):
                break
        last_height = new_height
    
    htmlSource = driver.page_source
    return htmlSource

def linkArr(htmlSource):
    linkArr = []
    
    soup = BeautifulSoup(htmlSource, 'html.parser')
    links = soup.find_all(class_ = 'resultlink')

    for i in range(1, len(links), 2):
        if not links[i].get('href').split('/')[3] == 'video':
            linkArr.append(links[i].get('href'))
    
    return linkArr
    
def articleDataFrame(linkArr):
    df = pd.DataFrame(columns = ['Year', 'Month', 'Day', 'Article'])
    
    for index,link in enumerate(linkArr):
        htmlSource = requests.get(link)
        soup = BeautifulSoup(htmlSource.text, 'html.parser')
        articles = soup.find_all('div', class_ = 'group')
        words = ''
        
        for article in articles:
            words += article.text
        
        
        df.loc[index] = [link.split('/')[3], link.split('/')[4], link.split('/')[5], words.replace(u'\\xa0', u' ')]
        
        print('index', index, 'has done.')
        if(index > 5):
            break
    
    return df
# blockchainHtml = linkInfoCrawler('blockchain')
# linkDataFrame(blockchainHtml)

In [373]:
linkArr = linkArr(blockchainHtml)

In [374]:
df = articleDataFrame(linkArr)

index 0 has done.
index 1 has done.
index 2 has done.
index 3 has done.
index 4 has done.
index 5 has done.
index 6 has done.


In [376]:
df['Article'][0]

'China has launched the\xa0"National Blockchain and Distributed Accounting Technology Standardization Technical Committee."\xa0Huawei, Tencent, Baidu, Ant Financial and JD.com are among a group of companies and academics on the committee.The aim is likely to set some national standards for the technology.Last year, President Xi Jinping said the country needs to "seize the opportunities" in blockchain.China\'s major technology giants are among a group of organizations that have joined a national committee focused on creating standards for blockchain technology.\xa0The move is part of major push by the world\'s second-largest economy to become a leader in the nascent space after President Xi Jinping\xa0last year urged the country to "seize the opportunities" presented by the technology.\xa0E-commerce giant JD.com, telecommunications equipment maker Huawei, search firm Baidu, and Tencent, the owner of popular messaging app WeChat, are all part of the committee. Ant Financial, the affiliat